In [ ]:
import pandas as pd
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_colwidth', None)
import numpy as np
import matplotlib.pyplot as plt
import scipy
import scipy.stats as stats
import math
plt.style.use('fivethirtyeight')

In [ ]:
# Import relevant dataframes:

# main_table = pd.read_csv(r"C:\Users\Public\_Data\_Data\Latest_Version\Main_Data\Main_Table_01192020.csv")
# print(len(main_table))
# faculty_table = pd.read_csv(r"C:\Users\Public\_Data\_Data\Latest_Version\Faculty_Data\Faculty_Table_11222019.csv")
# print(len(faculty_table))
# skill_table = pd.read_csv(r"C:\Users\Public\_Data\_Data\Latest_Version\Skills_Data\Skill_Table_06072019.csv")
# print(len(skill_table))

def format_df(df):
    print(len(df))
    df = df.rename(columns={'BGTJobId':'Job ID'})
    return df

new_main = pd.read_parquet(r"A:\HEJP_revision_02152020\Jobs.parquet")
# new_main = format_df(new_main)

# new_cert = pd.read_parquet(r"A:\HEJP_revision_02152020\Certifications.parquet")
# new_cert = format_df(new_cert)

# new_skills = pd.read_parquet(r"A:\HEJP_revision_02152020\Skills.parquet")
# new_skills = format_df(new_skills)

# new_app = pd.read_parquet(r"A:\HEJP_revision_02152020\Faculty_Appointments.parquet")
# new_app = format_df(app)

# new_fac = pd.read_parquet(r"A:\HEJP_revision_02152020\Faculty_and_Postdoc_Fields.parquet")
# new_fac = format_df(new_fac)

In [ ]:
new_main

In [ ]:
new_main.groupby(['Occupation', 'Career Area']).count().reset_index()[['Occupation', 'Career Area']].groupby('Occupation').count().sort_values('Career Area')

In [ ]:
new_main.info()

In [ ]:
temp = new_skills.rename(columns={'Is Specialized Skill?':'Specialized', 
                                 'Is Software Skill?':'Software'})

In [ ]:
del(new_skills)

In [ ]:
# temp = new_skills

# temp = temp.convert_dtypes()

display(temp)

temp.info()

In [ ]:
temp.to_parquet(r"A:\HEJP_revision_02152020\Skills.parquet", index=False)

In [ ]:
del(temp)

In [ ]:
temp[:2].to_csv(r"A:\test.csv", index=False)

In [ ]:
test_in = pd.read_csv(r"A:\test.csv").convert_dtypes()

In [ ]:
test_in.info()

In [ ]:
pd.NA

In [ ]:
new_cert.info()

In [ ]:
new_cert.groupby(['BGTJobId', 'Certification Name']).count().sort_values(by='Year', ascending=False).reset_index()['Certification Name'].value_counts()

In [ ]:
new_fac['Others'].value_counts()

In [ ]:
temp = new_fac[(new_fac['Other_Engineering']==1)|(new_fac['Other_Education']==1)|(new_fac['Other_Social_Sciences']==1)|(new_fac['Other_Humanities_and_Arts']==1)]

temp['BGTJobId'].nunique()

In [ ]:
new_fac.columns

In [ ]:
for column in new_fac_det.columns:
    print(column)

In [ ]:
new_fac_det[new_fac_det['Other_Humanities_and_Arts']==1]

In [ ]:
new_fac_gen

In [ ]:
def validate_nas(df):
    
    if 'Faculty' not in df.columns:
        df = df.merge(faculty_table[['Job ID', 'Faculty']], on='Job ID', how='inner')
    
    nas = df[df['Occupation']=='na']
    no_nas = df.drop(nas.index)
    
    n_w = nas.iloc[:,0].nunique()
    n_wo = no_nas.iloc[:,0].nunique()
    
    def percent(s_df):
        s_df['%nas'] = s_df['nas'] / (s_df['nas'] + s_df['no_nas'])
        return s_df
    
    total = pd.DataFrame([[n_w, n_wo]], columns=['nas', 'no_nas'])
    total = percent(total)
    
    display(total)
    
    years_w = pd.DataFrame(nas['Year'].value_counts()).reset_index().rename(columns={'Year':'nas'})
    years_wo = pd.DataFrame(no_nas['Year'].value_counts()).reset_index().rename(columns={'Year':'no_nas'})
    
    years = years_w.merge(years_wo, on='index', how='inner').rename(columns={'index':'Year'})
    years = years.sort_values(by='Year').reset_index(drop=True)
    years = percent(years)
    
    display(years)
        
    fac_w = pd.DataFrame(nas['Faculty'].value_counts()).reset_index().rename(columns={'Faculty':'nas'})
    fac_wo = pd.DataFrame(no_nas['Faculty'].value_counts()).reset_index().rename(columns={'Faculty':'no_nas'})
    
    fac = fac_w.merge(fac_wo, on='index', how='inner').rename(columns={'index':'Faculty'})
    fac = percent(fac)
    
    display(fac)
    
validate_nas(main_table)
validate_nas(new_main)

In [ ]:
new_main = new_main.sort_values(by='BGTJobId')
new_main = new_main.rename(columns={'BGTJobId':'Job ID'})

In [ ]:
together = main_table.merge(new_main[['Job ID']], on='Job ID', how='outer', indicator=True)

In [ ]:
pd.DataFrame(together.groupby('_merge')[['Job ID']].count())

In [ ]:
main_indi = together[~together['Occupation'].isnull()]

def in_or_not(string):
    if string is 'left_only':
        return 0
    elif string is 'both':
        return 1
    else:
        return None
    
main_indi['kept'] = main_indi['_merge'].apply(in_or_not)

main_indi = main_indi.merge(faculty_table[['Job ID', 'Faculty', 'Tenure_Line']])

both = main_indi[main_indi['kept']==1]
left = main_indi[main_indi['kept']==0]

right = new_main.merge(together[together['_merge']=='right_only'][['Job ID']], on='Job ID', how='inner')

In [ ]:
display(stats.ttest_ind(both['Faculty'], left['Faculty'], equal_var=True))
display(both['Faculty'].describe())
display(left['Faculty'].describe())

In [ ]:
left

In [ ]:
len(left[~left['IPEDS Institution Name'].isnull()])

In [ ]:
len(left[left['IPEDS Institution Name'].isnull()])/638330

In [ ]:
0.033280591543558974 * 638330

In [ ]:
len(both[both['IPEDS Institution Name'].isnull()&both['IPEDS Sector Name'].isnull()])

In [ ]:
len(right[right['IPEDS Institution Name'].isnull()&right['IPEDS Sector Name'].isnull()])

In [ ]:
0.10744908532717937 * 3318837 + 0.10199166891498689 * 3306172

In [ ]:
75813 + 303998

In [ ]:
693808 - 379811

In [ ]:
display(stats.ttest_ind(both['Faculty'], right['Faculty'], equal_var=True))
display(both['Faculty'].describe())
display(right['Faculty'].describe())

In [ ]:
new_main

In [ ]:
new_cert.groupby('Certification Name').count().sort_values('Job ID', ascending=False)

In [ ]:
right